In [1]:
%reload_ext autoreload
%autoreload 2

# Assignment

Hello!

In this assignment, you'll be given a dataset you don't know a lot about, and you'll be asked to train a **regressor**, that is a model that estimates a **real-valued number** given some input _features_.

You will find a **training set**, consisting of 11 features and one target, and a **test set**, where the target is missing.

The goal is to use the training set **however you want**, and then use the trained model to predict the target for the test set.
You won't be able to determine the performance of the model on the test set, since you're missing the target for that, so you'll have to do your best to make sure that your model generalizes well on unseen data points.

You're expected to produce a csv file with only **one** column (no header) containing all the predictions on the test set, then submit it somewhere (still working on where/how at the time of writing these instructions), and I will evaluate your results (since I have the target for the test set).

Some examples are included, as well as some checks to make sure that the output file has the expected format.

In [2]:
import pandas as pd

In [3]:
from pathlib import Path

In [4]:
DATA_FOLDER = "../data/challenge"

TRAIN_CSV = Path(DATA_FOLDER) / "challenge_train.csv"
TEST_CSV = Path(DATA_FOLDER) / "challenge_test.csv"

In [5]:
df_train = pd.read_csv(TRAIN_CSV)
df_train.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,target
0,329.606668,-114.715869,-28.435582,3236.787208,-397.455876,956.789861,681.827073,1.958500,0.532288,7.006549,prometheus,81695.585876
1,335.950586,-116.887718,-81.515335,4219.908247,-440.501638,1477.429690,998.676125,1.501246,0.452003,7.445087,prometheus,84468.284549
2,335.978288,-114.082413,-18.957055,30065.213170,-2545.439435,13552.620106,6709.980549,2.015428,0.611648,5.981045,prometheus,118235.793378
3,324.869508,-100.146381,-64.453986,6048.970645,-462.024520,2233.270845,1331.568166,2.404268,1.197221,3.540756,enterprise,155370.150594
4,331.656668,-103.856623,-62.558280,3357.635974,-540.941751,1719.481540,1427.826106,1.295065,0.788453,5.991467,enterprise,241422.834384


In [ ]:
df_test = pd.read_csv(TEST_CSV)
df_test.head()

# You might need to do this if you use some specific algorithm
df_train['feature_11'] = df_train['feature_11'].astype("category")
df_test['feature_11'] = df_test['feature_11'].astype("category")

## Examples

In [9]:
# A silly thing you can do, which is better than nothing, is to always predict the same value, 
# the mean of the target computed on the training set
y_hat_dumb = len(df_test)*[df_train['target'].mean()]

y_hat = y_hat_dumb

In [10]:
# This should make sure that your data has the right format
# If you see an error, something is wrong
assert len(y_hat) == len(df_test)

In [12]:
# Save file somewhere to disk
pd.DataFrame(y_hat).to_csv("test_predictions_MYNAMEORGROUPNAME.csv", header=False, index=False)